# Monitoramento de Marcos Topográficos 🚀
Este notebook lê dados de um arquivo `.txt`, organiza segundo ordem fixa de marcos, e atualiza uma planilha Excel preenchendo coordenadas iniciais em cada aba correspondente ao marco.

### Como usar
1. Faça upload do `dados.txt` e `monitoramento.xlsx`.
2. Execute as células para processar os dados.
3. Baixe o arquivo `monitoramento_atualizado.xlsx`.


In [ ]:
# Instala bibliotecas necessárias
!pip install openpyxl pandas --quiet

In [ ]:
from google.colab import files
print("📁 Faça upload dos arquivos:")
print("- dados.txt")
print("- monitoramento.xlsx")
uploaded = files.upload()

In [ ]:
import pandas as pd
from openpyxl import load_workbook

# Ordem fixa dos marcos
ordem_marcos = [
    "SVN-CV-02", "SVN-CV-19", "SVN-CV-09", "SVN-CV-22", "SVN-CV-28",
    "SVN-CV-29", "SVN-CV-31", "SVN-CV-35", "SVN-CV-37", "SVN-CV-38",
    "SVN-CV-39", "SVN-CV-40", "SVN-CV-41", "SVN-CV-42", "SVN-CV-43",
    "SVN-CV-45", "SVN-CV-46", "SVN-CV-47", "SVN-CV-48", "SVN-CV-49",
    "SVN-CV-50", "SVN-CV-51", "SVN-CV-32", "SVN-CV-33", "SVN-CV-34",
    "SVN-CV-27", "SVN-CV-52", "SVN-CV-53"
]

# Leitura do TXT (pulando cabeçalho BASE‑SVN)
with open("dados.txt", "r", encoding="utf-8") as f:
    linhas = f.readlines()[1:]

dados = []
for linha in linhas:
    partes = linha.strip().split(",")
    if len(partes) == 5:
        _, este, norte, cota, marco = partes
        try:
            este_f = float(este)
            norte_f = float(norte)
            cota_f = float(cota)
        except:
            # se algum valor não for float, ignora
            continue
        dados.append({
            "marco": marco.strip(),
            "este": este_f,
            "norte": norte_f,
            "cota": cota_f
        })

df = pd.DataFrame(dados)

# Ordena pela ordem fixa
df["ordem"] = df["marco"].apply(lambda x: ordem_marcos.index(x) if x in ordem_marcos else 999)
df = df.sort_values("ordem").reset_index(drop=True)

# Exibe para visualização (opcional)
df

In [ ]:
# Atualiza Excel nas abas correspondentes
wb = load_workbook("monitoramento.xlsx")

# Defina a linha onde vai inserir os dados iniciais (ex: 310)
linha_para_preencher = 310

for _, linha in df.iterrows():
    nome_aba = linha["marco"]
    if nome_aba in wb.sheetnames:
        ws = wb[nome_aba]
        ws.cell(row=linha_para_preencher, column=3, value=linha["este"])   # Coluna C
        ws.cell(row=linha_para_preencher, column=5, value=linha["norte"])  # Coluna E
        ws.cell(row=linha_para_preencher, column=7, value=linha["cota"])   # Coluna G
    else:
        print(f"⚠️ Aba não encontrada: {nome_aba}")

# Salva o arquivo novo
saida = "monitoramento_atualizado.xlsx"
wb.save(saida)
print(f"✅ Planilha salva como '{saida}'")

In [ ]:
from google.colab import files
files.download("monitoramento_atualizado.xlsx")